In [2]:
from selenium import webdriver

import pandas as pd
import time
from numpy import random
import pickle

from bs4 import BeautifulSoup

def flatten(t):
    return [item for sublist in t for item in sublist]

In [2]:
def chrome(path, headless=True):
    "Chrome driver"
    opt = webdriver.ChromeOptions()
    if headless:
        opt.add_argument("--headless")
    opt.add_argument('--blink-settings=imagesEnabled=false')
    browser = webdriver.Chrome(path, options=opt)
    browser.implicitly_wait(1)
    browser.set_page_load_timeout(3)

    return browser

In [5]:
driver = chrome('C:/Users/59898/Documents/chromedriver.exe', headless=True)

In [16]:
urls= ['https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag={}'.format(i) for i in range(1, 3)]

In [32]:
data = []
metraje = []
url_publicacion = []

for url in urls:
    try:
        driver.get(url)
    except:
        print('Get method failed, check URL.')
    pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
    data.append(pagina)
    html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]
    for i in html:
        soup = BeautifulSoup(i, "html.parser")
        metraje.append(soup.span.text)
        url_publicacion.append(soup.find('a')['href'])
flat_list = flatten(data)
result = pd.DataFrame(flat_list, columns=['desc', 'valor'])
result['metraje'] = metraje
result['metraje'] = result['metraje'].replace({' m2': ''}, regex=True)
result['url'] = url_publicacion

In [24]:
# coordenadas = []

# for url in urls:
#     driver.get(url)

#     elem = driver.find_elements_by_css_selector('#ubicacion')

#     if elem != []:
#         element = driver.find_element_by_xpath("/html/body/form/main/div/div[1]/ul/li[4]/a/i")
#         element.click()
#         time.sleep(2)
#         ubic_html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector("#ubicacion")]
#         soup = BeautifulSoup(ubic_html[0], "html.parser")
#         src = soup.find('iframe')['src']
#         latlng = src.split('q=', 1)[1].split('&zoom=', 1)[0]
#         coordenadas.append(latlng)
#     else:
#         print('Sin ubicación')
#         coordenadas.append('')

In [7]:
def scrapper(urls):
    data = []
    metraje = []
    url_publicacion=[]

    for url in urls:
        try:
            driver.get(url)
            time.sleep(random.uniform(0.9, 2.3))
        except:
            print('Get method failed, check URL.')
        pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
        data.append(pagina)
        html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]

        for i in html:
            soup = BeautifulSoup(i, "html.parser")
            metraje.append(soup.span.text)
            url_publicacion.append(soup.find('a')['href'])

    flat_list = flatten(data)
    result = pd.DataFrame(flat_list, columns=['desc', 'valor'])
    result['metraje'] = metraje
    result['metraje'] = result['metraje'].replace({' m2': ''}, regex=True)
    result['url'] = url_publicacion

    return result

In [8]:
urls= [f'https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag={i}' for i in range(1, 143)]

data = scrapper(urls)

driver.quit()

In [9]:
data.head()

,desc,valor,metraje,url
0,Más de 4 dormitorios en Pocitos,U$S 1,,https://www.gallito.com.uy/se-vende-casa-para-...
1,2 Dormitorios en Belvedere,U$S 30.000,55,https://www.gallito.com.uy/casa-a-reciclar-lug...
2,2 Dormitorios en La Teja,U$S 35.000,76,https://www.gallito.com.uy/sp-curuzu-la-teja-i...
3,3 Dormitorios en Paso De La Arena,U$S 39.000,74,https://www.gallito.com.uy/2-casas-en-una-mism...
4,3 Dormitorios en Cerrito,U$S 40.000,132,https://www.gallito.com.uy/dueño-vende-2-casas...


In [12]:
data.to_csv('ventas_mvdeo_20221701.csv', index=False)

## Recupera coordenadas

In [3]:
data = pd.read_csv('ventas_mvdeo_20221701.csv')
data.head()

,desc,valor,metraje,url
0,Más de 4 dormitorios en Pocitos,U$S 1,NaN,https://www.gallito.com.uy/se-vende-casa-para-...
1,2 Dormitorios en Belvedere,U$S 30.000,55.0,https://www.gallito.com.uy/casa-a-reciclar-lug...
2,2 Dormitorios en La Teja,U$S 35.000,76.0,https://www.gallito.com.uy/sp-curuzu-la-teja-i...
3,3 Dormitorios en Paso De La Arena,U$S 39.000,74.0,https://www.gallito.com.uy/2-casas-en-una-mism...
4,3 Dormitorios en Cerrito,U$S 40.000,132.0,https://www.gallito.com.uy/dueño-vende-2-casas...


In [4]:
def scrap_latlng(urls):
    coordenadas = []

    for url in urls:
        try:
            driver.get(url)
            elem = driver.find_elements_by_css_selector('#ubicacion')
        except:
            try:
                time.sleep(5)
                driver.get(url)
                elem = driver.find_elements_by_css_selector('#ubicacion')
            except:
                elem = []
    
        if elem != []:
            element = driver.find_element_by_xpath("/html/body/form/main/div/div[1]/ul/li[4]/a/i")
            element.click()
            ubic_html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector("#ubicacion")]
            soup = BeautifulSoup(ubic_html[0], "html.parser")
            src = soup.find('iframe')['src']
            latlng = src.split('q=', 1)[1].split('&zoom=', 1)[0]
            if latlng == '/,/':
                coordenadas.append('Nan,Nan')
            else:
                coordenadas.append(latlng)
        else:
            coordenadas.append('Nan,Nan')

    return coordenadas

In [5]:
urls = list(data.url.values)
len(urls)

5680

In [30]:
urls[0:5]

['https://www.gallito.com.uy/se-vende-casa-para-familia-o-empresa-pocitos-inmuebles-21050486',
 'https://www.gallito.com.uy/casa-a-reciclar-lugar-para-vehiculo-imperdible-inmuebles-21097237',
 'https://www.gallito.com.uy/sp-curuzu-la-teja-inmuebles-19759592',
 'https://www.gallito.com.uy/2-casas-en-una-misma-unidad-en-paso-de-la-arena-inmuebles-20925137',
 'https://www.gallito.com.uy/dueño-vende-2-casas-en-padron-unico-para-reciclar-inmuebles-20783605']

In [7]:
driver = chrome('C:/Users/59898/Documents/chromedriver.exe', headless=True)

# CORRE EL SCRAPPER
coords = scrap_latlng(urls)

# guarda en in pickle
with open('coords.pkl', 'wb') as f:
    pickle.dump(coords, f)

In [8]:
driver.quit()

### Levanta las coodenadas

In [11]:
data = pd.read_csv('ventas_mvdeo_20221701.csv')
data.head()

,desc,valor,metraje,url
0,Más de 4 dormitorios en Pocitos,U$S 1,NaN,https://www.gallito.com.uy/se-vende-casa-para-...
1,2 Dormitorios en Belvedere,U$S 30.000,55.0,https://www.gallito.com.uy/casa-a-reciclar-lug...
2,2 Dormitorios en La Teja,U$S 35.000,76.0,https://www.gallito.com.uy/sp-curuzu-la-teja-i...
3,3 Dormitorios en Paso De La Arena,U$S 39.000,74.0,https://www.gallito.com.uy/2-casas-en-una-mism...
4,3 Dormitorios en Cerrito,U$S 40.000,132.0,https://www.gallito.com.uy/dueño-vende-2-casas...


In [12]:
coords = pd.read_pickle('coords.pkl')

len(coords)

5680

In [13]:
r = [i=='Nan,Nan' for i in coords]
sum(r)

1970

In [14]:
data[['lat', 'lng']] = [i.split(',') for i in coords]

In [15]:
data['lat'] = data['lat'].astype(float).round(5)
data['lng'] = data['lng'].astype(float).round(5)

In [16]:
data.head(8)

,desc,valor,metraje,url,lat,lng
0,Más de 4 dormitorios en Pocitos,U$S 1,NaN,https://www.gallito.com.uy/se-vende-casa-para-...,-34.90887,-56.15031
1,2 Dormitorios en Belvedere,U$S 30.000,55.0,https://www.gallito.com.uy/casa-a-reciclar-lug...,NaN,NaN
2,2 Dormitorios en La Teja,U$S 35.000,76.0,https://www.gallito.com.uy/sp-curuzu-la-teja-i...,-34.87447,-56.23885
3,3 Dormitorios en Paso De La Arena,U$S 39.000,74.0,https://www.gallito.com.uy/2-casas-en-una-mism...,-34.90721,-56.19027
4,3 Dormitorios en Cerrito,U$S 40.000,132.0,https://www.gallito.com.uy/dueño-vende-2-casas...,-34.85083,-56.15761
5,1 Dormitorio en Malvin Norte,U$S 40.000,65.0,https://www.gallito.com.uy/oportunidad-saldo-c...,-34.90721,-56.19027
6,4 Dormitorios en Cerrito,U$S 53.000,97.0,https://www.gallito.com.uy/oportunidad-casa-de...,-34.85944,-56.15642
7,1 Dormitorio en Prado Norte,U$S 59.000,50.0,https://www.gallito.com.uy/impecable-buena-vec...,NaN,NaN


In [17]:
data.to_csv('ventas_mvdeo_20220117_con_coord.csv', index=False)

In [18]:
# limpieza
data['valor'].str.replace('U', '')

for word, rep in {"U":" ", "S":"", "$":"", ".":""}.items():
    data['valor'] = data['valor'].str.replace(word, rep, regex=False)

In [19]:
data['valor'] = data['valor'].astype(int)

In [20]:
data = data.loc[~((data.valor == 111111111) | (data.valor < 45000))]
data.head()

,desc,valor,metraje,url,lat,lng
6,4 Dormitorios en Cerrito,53000,97.0,https://www.gallito.com.uy/oportunidad-casa-de...,-34.85944,-56.15642
7,1 Dormitorio en Prado Norte,59000,50.0,https://www.gallito.com.uy/impecable-buena-vec...,NaN,NaN
8,2 Dormitorios en Maroñas,65000,NaN,https://www.gallito.com.uy/muy-linda-casa-y-co...,-34.90721,-56.19027
9,2 Dormitorios en Union,65000,103.0,https://www.gallito.com.uy/venta-casa-union-2-...,-34.87518,-56.13185
10,2 Dormitorios en La Teja,67000,NaN,https://www.gallito.com.uy/dueño-vende-casa-en...,-34.87064,-56.23490


In [21]:
data[['dormitorios', 'barrio']] = data['desc'].str.split(' en ', n = 1, expand = True)

In [22]:
data['barrio_ine'] = data['barrio']

In [23]:
def format_barrio(df, column):
    "Formatea strings de Barrio Gallito a Barrio INE"
    barrios_dict = {
        'Aguada' : 'Aguada',
        'Aires Puros' : 'Aires Puros',
        'Atahualpa' : 'Atahualpa',
        'B. De Carrasco' : 'Bañados de Carrasco',
        'Barrio Sur' : 'Barrio Sur',
        'Belvedere' : 'Belvedere',
        'Brazo Oriental' : 'Brazo Oriental',
        'Buceo' : 'Buceo',
        'Capurro' : 'Capurro, Bella Vista',
        'Carrasco' : 'Carrasco',
        'Carrasco Norte' : 'Carrasco Norte',
        'Casabo' : 'Casabó, Pajas Blancas',
        'Casavalle' : 'Casavalle',
        'Centro' : 'Centro',
        'Cerrito' : 'Cerrito',
        'Cerro' : 'Cerro',
        'Ciudad Vieja' : 'Ciudad Vieja',
        'Cno. Maldonado' : 'Colón Centro y Noroeste',
        'Colon' : 'Colón Sureste, Abayubá',
        'Cordon' : 'Cordón',
        'Golf' : 'Flor de Maroñas',
        'Ituzaingo' : 'Ituzaingó',
        'J. Hipodromo' : 'Jardines del Hipódromo',
        'Jacinto Vera' : 'Jacinto Vera',
        'La Blanqueada' : 'La Blanqueada',
        'La Comercial' : 'La Comercial',
        'La Figurita' : 'La Figurita',
        'La Teja' : 'La Teja',
        'Larrañaga' : 'Larrañaga',
        'Las Acacias' : 'Las Acacias',
        'Lezica' : 'Lezica, Melilla',
        'Malvin' : 'Malvín',
        'Malvin Norte' : 'Malvín Norte',
        'Manga' : 'Manga',
        'Maroñas' : 'Maroñas, Parque Guaraní',
        'Maroñas Curva' : 'Maroñas, Parque Guaraní',
        'Melilla' : 'Lezica, Melilla',
        'Nuevo Paris' : 'Mercado Modelo, Bolívar',
        'Pajas Blancas' : 'Nuevo París',
        'Palermo' : 'Palermo',
        'Parque Batlle' : 'Parque Batlle, Villa Dolores',
        'Parque Rodo' : 'Parque Rodó',
        'Paso De La Arena' : 'Paso de la Arena',
        'Peñarol' : 'Peñarol, Lavalleja',
        'Perez Castellanos' : 'Castro, P. Castellanos',
        'Piedras Blancas' : 'Piedras Blancas',
        'Pocitos' : 'Pocitos',
        'Pocitos Nuevo' : 'Pocitos',
        'Prado' : 'Prado, Nueva Savona',
        'Prado Norte' : 'Prado, Nueva Savona',
        'Puerto Buceo' : 'Buceo',
        'Punta Carretas' : 'Punta Carretas',
        'Punta Gorda' : 'Punta Gorda',
        'Punta Rieles' : 'Punta Rieles, Bella Italia',
        'Reducto' : 'Reducto',
        'Sayago' : 'Sayago',
        'Tres Cruces' : 'Tres Cruces',
        'Union' : 'Unión',
        'Villa Dolores' : 'Parque Batlle, Villa Dolores',
        'Villa Española' : 'Villa Española',
        'Villa Muñoz' : 'Villa Muñoz, Retiro'
        }

    return df[column].map(barrios_dict)

In [24]:
data['barrio_ine'] = format_barrio(data, 'barrio')

In [26]:
def encode_barrio(df, column):
    "Codifica barrio INE"
    barrios_dict = {
    'Ciudad Vieja':1,
    'Centro':2,
    'Barrio Sur':3,
    'Cordón':4,
    'Palermo':5,
    'Parque Rodó':6,
    'Punta Carretas':7,
    'Pocitos':8,
    'Buceo':9,
    'Parque Batlle, Villa Dolores':10,
    'Malvín':11,
    'Malvín Norte':12,
    'Punta Gorda':13,
    'Carrasco':14,
    'Carrasco Norte':15,
    'Bañados de Carrasco':16,
    'Maroñas, Parque Guaraní':17,
    'Flor de Maroñas':18,
    'Las Canteras':19,
    'Punta Rieles, Bella Italia':20,
    'Jardines del Hipódromo':21,
    'Ituzaingó':22,
    'Unión':23,
    'Villa Española':24,
    'Mercado Modelo, Bolívar':25,
    'Castro, P. Castellanos':26,
    'Cerrito':27,
    'Las Acacias':28,
    'Aires Puros':29,
    'Casavalle':30,
    'Piedras Blancas':31,
    'Manga, Toledo Chico':32,
    'Paso de las Duranas':33,
    'Peñarol, Lavalleja':34,
    'Cerro':35,
    'Casabó, Pajas Blancas':36,
    'La Paloma, Tomkinson':37,
    'La Teja':38,
    'Prado, Nueva Savona':39,
    'Capurro, Bella Vista':40,
    'Aguada':41,
    'Reducto':42,
    'Atahualpa':43,
    'Jacinto Vera':44,
    'La Figurita':45,
    'Larrañaga':46,
    'La Blanqueada':47,
    'Villa Muñoz, Retiro':48,
    'La Comercial':49,
    'Tres Cruces':50,
    'Brazo Oriental':51,
    'Sayago':52,
    'Conciliación':53,
    'Belvedere':54,
    'Nuevo París':55,
    'Tres Ombúes, Victoria':56,
    'Paso de la Arena':57,
    'Colón Sureste, Abayubá':58,
    'Colón Centro y Noroeste':59,
    'Lezica, Melilla':60,
    'Villa García, Manga Rural':61,
    'Manga':62
        }
    return df[column].map(barrios_dict)

In [ ]:
data['cod_barrio_ine'] = encode_barrio(data, 'barrio_ine')

In [28]:
data.to_csv('ventas_mvdeo_20220117_depurado.csv', index=False)

In [29]:
data = data.loc[(data.metraje > 10) & (data.metraje < 4000) & (~data.metraje.isna())]

data['metraje'] = data['metraje'].astype(int)

data['valor_metro'] = data['valor'] / data['metraje']

agru = data.groupby('cod_barrio_ine').median('valor_metro').reset_index()

agru['cod_barrio_ine'] = agru['cod_barrio_ine'].astype(int)
                                                       
agru.to_csv('datos_agrupados_20220117.csv', index=False)